# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:

# Load the csv exported in Part I to a DataFrame

weather_csv_file = "../output_data/city_weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,79.11,68.0,100.0,17.56,PF,1.619215e+09
1,tessalit,20.1986,1.0114,88.03,7.0,0.0,6.64,ML,1.619215e+09
2,bandarbeyla,9.4942,50.8122,82.92,73.0,100.0,7.38,SO,1.619215e+09
3,barrow,71.2906,-156.7887,19.40,79.0,1.0,3.44,US,1.619215e+09
4,salalah,17.0151,54.0924,82.40,74.0,40.0,5.75,OM,1.619215e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "lng"]].astype(float)

# Create a humidity Heatmap layer
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,mar del plata,-38.0023,-57.5575,71.60,78.0,0.0,6.91,AR,1.619214e+09
203,noumea,-22.2763,166.4572,79.00,88.0,0.0,2.30,NC,1.619215e+09
216,namibe,-15.1961,12.1522,78.22,77.0,0.0,4.14,AO,1.619215e+09
260,bam,29.1060,58.3570,79.02,17.0,0.0,9.64,IR,1.619215e+09
269,sakakah,29.9697,40.2064,78.80,11.0,0.0,8.05,SA,1.619215e+09
305,ibra,22.6906,58.5334,76.12,33.0,0.0,0.29,OM,1.619215e+09
445,asyut,27.1810,31.1837,76.32,19.0,0.0,9.42,EG,1.619215e+09
468,rudbar,30.1500,62.6000,72.46,11.0,0.0,6.69,AF,1.619215e+09
492,sao joao da barra,-21.6403,-41.0511,77.00,65.0,0.0,9.22,BR,1.619215e+09
515,joao pinheiro,-17.7425,-46.1725,70.90,43.0,0.0,2.15,BR,1.619215e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 58: mar del plata.
Missing field/result... skipping.
------------
Retrieving Results for Index 203: noumea.
Closest hotel is Hôtel Le Lagon.
------------
Retrieving Results for Index 216: namibe.
Closest hotel is Hotel Chik Chik Namibe.
------------
Retrieving Results for Index 260: bam.
Closest hotel is Bam Tourist Hotel.
------------
Retrieving Results for Index 269: sakakah.
Missing field/result... skipping.
------------
Retrieving Results for Index 305: ibra.
Closest hotel is مزرعة راحة النفوس المنزفة.
------------
Retrieving Results for Index 445: asyut.
Closest hotel is Florence Hotel Assiut.
------------
Retrieving Results for Index 468: rudbar.
Missing field/result... skipping.
------------
Retrieving Results for Index 492: sao joao da barra.
Missing field/result... skipping.
------------
Retrieving Results for Index 515: joao pinheiro.
Closest hotel is Hotel Santos.
------------
Retrieving Results for Index 541: paso de carrasco.
Closest hotel is H

In [14]:
hotel_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
58,mar del plata,-38.0023,-57.5575,71.60,78.0,0.0,6.91,AR,1.619214e+09,NaN
203,noumea,-22.2763,166.4572,79.00,88.0,0.0,2.30,NC,1.619215e+09,Hôtel Le Lagon
216,namibe,-15.1961,12.1522,78.22,77.0,0.0,4.14,AO,1.619215e+09,Hotel Chik Chik Namibe
260,bam,29.1060,58.3570,79.02,17.0,0.0,9.64,IR,1.619215e+09,Bam Tourist Hotel
269,sakakah,29.9697,40.2064,78.80,11.0,0.0,8.05,SA,1.619215e+09,NaN
305,ibra,22.6906,58.5334,76.12,33.0,0.0,0.29,OM,1.619215e+09,مزرعة راحة النفوس المنزفة
445,asyut,27.1810,31.1837,76.32,19.0,0.0,9.42,EG,1.619215e+09,Florence Hotel Assiut
468,rudbar,30.1500,62.6000,72.46,11.0,0.0,6.69,AF,1.619215e+09,NaN
492,sao joao da barra,-21.6403,-41.0511,77.00,65.0,0.0,9.22,BR,1.619215e+09,NaN
515,joao pinheiro,-17.7425,-46.1725,70.90,43.0,0.0,2.15,BR,1.619215e+09,Hotel Santos


In [15]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Lat", "lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))